In [1]:
# Imports
import pandas as pd
import numpy as np
import requests
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports réussis!")

✅ Imports réussis!


In [2]:
# Récupération des données historiques (2 ANS pour mieux capturer la saisonnalité)
print("📥 Téléchargement des données historiques (2 ANS)...")
print("⏳ Cela peut prendre quelques secondes...")

end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")  # 2 ans

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 34.0209,
    "longitude": -6.8416,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m,cloud_cover,pressure_msl",
    "daily": "temperature_2m_max,temperature_2m_min",
    "timezone": "GMT"
}

response = requests.get(url, params=params)
data = response.json()

print(f"✅ Données récupérées du {start_date} au {end_date}")
print(f"📊 {len(data['hourly']['time'])} heures de données téléchargées")

📥 Téléchargement des données historiques (2 ANS)...
⏳ Cela peut prendre quelques secondes...
✅ Données récupérées du 2023-12-05 au 2025-12-04
📊 17544 heures de données téléchargées


In [3]:
# Préparation des données AVANCÉE
print("🔧 Préparation avancée des données...")

# DataFrame horaire avec toutes les variables météo
df = pd.DataFrame({
    'date': pd.to_datetime(data['hourly']['time']),
    'temperature': data['hourly']['temperature_2m'],
    'humidity': data['hourly']['relative_humidity_2m'],
    'wind_speed': data['hourly']['wind_speed_10m'],
    'cloud_cover': data['hourly']['cloud_cover'],
    'pressure': data['hourly']['pressure_msl']
})

# Features temporelles
df['jour_annee'] = df['date'].dt.dayofyear
df['mois'] = df['date'].dt.month
df['jour_semaine'] = df['date'].dt.dayofweek
df['heure'] = df['date'].dt.hour

# Encodage cyclique COMPLET (capture les cycles naturels)
df['day_cos'] = np.cos(2 * np.pi * df['jour_annee'] / 365.25)
df['day_sin'] = np.sin(2 * np.pi * df['jour_annee'] / 365.25)
df['hour_cos'] = np.cos(2 * np.pi * df['heure'] / 24)
df['hour_sin'] = np.sin(2 * np.pi * df['heure'] / 24)
df['month_cos'] = np.cos(2 * np.pi * df['mois'] / 12)
df['month_sin'] = np.sin(2 * np.pi * df['mois'] / 12)

# Suppression des valeurs nulles
df = df.dropna()

print(f"✅ {len(df)} observations préparées")
print(f"📊 Variables: température, humidité, vent, nuages, pression")
print(f"📊 Période: {df['date'].min().strftime('%Y-%m-%d')} → {df['date'].max().strftime('%Y-%m-%d')}")
df.describe()

🔧 Préparation avancée des données...
✅ 17544 observations préparées
📊 Variables: température, humidité, vent, nuages, pression
📊 Période: 2023-12-05 → 2025-12-04


,date,temperature,humidity,wind_speed,cloud_cover,pressure,jour_annee,mois,jour_semaine,heure,day_cos,day_sin,hour_cos,hour_sin,month_cos,month_sin
count,17544,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,17544.000000,1.754400e+04,1.754400e+04,1.754400e+04,1.754400e+04
mean,2024-12-04 11:30:00,18.981025,76.523313,9.901972,44.648769,1017.766912,183.250342,6.519836,2.995896,11.500000,0.000684,0.000013,-5.558709e-17,-1.518773e-17,-1.685432e-03,-4.189026e-03
min,2023-12-05 00:00:00,5.300000,15.000000,0.000000,0.000000,1001.200000,1.000000,1.000000,0.000000,0.000000,-0.999979,-0.999999,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,2024-06-04 17:45:00,15.600000,67.000000,5.900000,2.000000,1014.300000,92.000000,4.000000,1.000000,5.750000,-0.707487,-0.703677,-7.071068e-01,-7.071068e-01,-8.660254e-01,-8.660254e-01
50%,2024-12-04 11:30:00,19.100000,79.000000,8.800000,33.000000,1017.200000,183.000000,7.000000,3.000000,11.500000,0.001075,-0.004301,-6.123234e-17,6.123234e-17,-1.836970e-16,-2.449294e-16
75%,2025-06-05 05:15:00,22.200000,88.000000,12.800000,95.000000,1020.900000,275.000000,10.000000,5.000000,17.250000,0.712031,0.706727,7.071068e-01,7.071068e-01,8.660254e-01,5.000000e-01
max,2025-12-04 23:00:00,39.300000,100.000000,50.000000,100.000000,1034.000000,366.000000,12.000000,6.000000,23.000000,0.999991,0.999986,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
std,NaN,4.755602,14.798821,5.799080,41.085237,5.102613,105.513966,3.449649,1.997657,6.922384,0.707368,0.706885,7.071269e-01,7.071269e-01,7.083331e-01,7.059043e-01


In [4]:
# Entraînement du modèle OPTIMISÉ
print("🧠 Entraînement du modèle Gradient Boosting optimisé...")
print("⏳ Cela peut prendre 1-2 minutes...")

# Features pour la prédiction
features = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'month_cos', 'month_sin', 
            'heure', 'mois', 'jour_annee']

X = df[features]
y = df['temperature']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modèle Gradient Boosting (meilleur que Random Forest pour les séries temporelles)
model = GradientBoostingRegressor(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.08,
    min_samples_split=5,
    min_samples_leaf=3,
    subsample=0.9,
    random_state=42
)

model.fit(X_train, y_train)

# Évaluation sur le test set
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n✅ Modèle entraîné avec succès!")
print(f"📊 Erreur Absolue Moyenne (MAE): {mae:.2f}°C")
print(f"📊 RMSE: {rmse:.2f}°C")

🧠 Entraînement du modèle Gradient Boosting optimisé...
⏳ Cela peut prendre 1-2 minutes...

✅ Modèle entraîné avec succès!
📊 Erreur Absolue Moyenne (MAE): 1.62°C
📊 RMSE: 2.39°C


In [5]:
# Sauvegarde du modèle ET des features
import os

save_path = r'C:\Users\User\OneDrive\Bureau\projet1111\cerveau_meteo_long_terme.pkl'

# On sauvegarde le modèle ET la liste des features
model_data = {
    'model': model,
    'features': features
}
joblib.dump(model_data, save_path)

print(f"💾 Modèle sauvegardé: {save_path}")
print(f"📋 Features sauvegardées: {features}")

if os.path.exists(save_path):
    print("✅ Fichier créé avec succès!")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\OneDrive\\Bureau\\projet1111\\cerveau_meteo_long_terme.pkl'

In [ ]:
# 🎯 TEST FINAL: Comparaison avec l'API Météo Officielle
print("🧪 Test du modèle vs API officielle (Open-Meteo)...")

# Récupérer les prévisions officielles pour les 7 prochains jours
url_forecast = "https://api.open-meteo.com/v1/forecast"
params_api = {
    "latitude": 34.0209, "longitude": -6.8416,
    "hourly": "temperature_2m",
    "timezone": "GMT",
    "forecast_days": 7
}
res = requests.get(url_forecast, params=params_api).json()

# Créer les prédictions IA pour les mêmes moments
df_test = pd.DataFrame({
    'date': pd.to_datetime(res['hourly']['time']),
    'API_Officielle': res['hourly']['temperature_2m']
})

# Préparer les features
df_test['jour_annee'] = df_test['date'].dt.dayofyear
df_test['mois'] = df_test['date'].dt.month
df_test['heure'] = df_test['date'].dt.hour
df_test['day_cos'] = np.cos(2 * np.pi * df_test['jour_annee'] / 365.25)
df_test['day_sin'] = np.sin(2 * np.pi * df_test['jour_annee'] / 365.25)
df_test['hour_cos'] = np.cos(2 * np.pi * df_test['heure'] / 24)
df_test['hour_sin'] = np.sin(2 * np.pi * df_test['heure'] / 24)
df_test['month_cos'] = np.cos(2 * np.pi * df_test['mois'] / 12)
df_test['month_sin'] = np.sin(2 * np.pi * df_test['mois'] / 12)

# Prédiction IA
df_test['Prediction_IA'] = model.predict(df_test[features])

# Calcul de l'écart
ecart_moyen = (df_test['Prediction_IA'] - df_test['API_Officielle']).abs().mean()
ecart_max = (df_test['Prediction_IA'] - df_test['API_Officielle']).abs().max()

print(f"\n" + "="*50)
print(f"🎯 RÉSULTATS DE LA COMPARAISON")
print(f"="*50)
print(f"📊 Écart MOYEN avec l'API officielle: {ecart_moyen:.2f}°C")
print(f"📊 Écart MAXIMUM: {ecart_max:.2f}°C")

# Afficher la comparaison à 14h (température max)
print(f"\n📅 Comparaison à 14h (pic de chaleur):")
df_14h = df_test[df_test['heure'] == 14][['date', 'API_Officielle', 'Prediction_IA']].copy()
df_14h['Écart'] = (df_14h['Prediction_IA'] - df_14h['API_Officielle']).round(1)
df_14h['API_Officielle'] = df_14h['API_Officielle'].round(1)
df_14h['Prediction_IA'] = df_14h['Prediction_IA'].round(1)
print(df_14h.to_string(index=False))

print(f"\n" + "="*50)
print(f"✅ MODÈLE OPTIMISÉ ET PRÊT!")
print(f"💡 Relancez app_meteo.py pour voir les résultats")
print(f"="*50)

🧪 Test du modèle vs API officielle (Open-Meteo)...

🎯 RÉSULTATS DE LA COMPARAISON
📊 Écart MOYEN avec l'API officielle: 3.08°C
📊 Écart MAXIMUM: 6.93°C

📅 Comparaison à 14h (pic de chaleur):
               date  API_Officielle  Prediction_IA  Écart
2025-12-04 14:00:00            18.1           19.4    1.3
2025-12-05 14:00:00            20.3           21.1    0.8
2025-12-06 14:00:00            20.1           20.2    0.1
2025-12-07 14:00:00            17.0           20.1    3.1
2025-12-08 14:00:00            20.0           21.0    1.0
2025-12-09 14:00:00            19.1           21.0    1.9
2025-12-10 14:00:00            16.1           20.2    4.1

✅ MODÈLE OPTIMISÉ ET PRÊT!
💡 Relancez app_meteo.py pour voir les résultats
